In [ ]:
import pytest
from unittest.mock import AsyncMock, Mock, patch
import httpx
from fh_saas.utils_api import AsyncAPIClient
from fh_saas.utils_graphql import GraphQLClient

## Test GraphQL Query Execution

In [ ]:
@pytest.mark.asyncio
async def test_execute_query_success():
    """Test successful GraphQL query execution"""
    
    # Mock response
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'data': {
            'users': [{'id': 1, 'name': 'Alice'}]
        }
    }
    mock_response.raise_for_status = Mock()
    
    # Mock API client
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            result = await client.execute_query(
                query='{ users { id name } }'
            )
            
            assert result == {'data': {'users': [{'id': 1, 'name': 'Alice'}]}}

# Run test
await test_execute_query_success()
print("✅ Test passed: Execute query success")

✅ Test passed: Execute query success


In [ ]:
@pytest.mark.asyncio
async def test_execute_query_with_errors():
    """Test GraphQL query with errors in response"""
    
    # Mock response with errors
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'errors': [{'message': 'Field not found'}]
    }
    mock_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            try:
                await client.execute_query('{ invalid }')
                assert False, "Should have raised ValueError"
            except ValueError as e:
                assert 'Field not found' in str(e)

# Run test
await test_execute_query_with_errors()
print("✅ Test passed: Execute query with errors")

✅ Test passed: Execute query with errors


## Test Pagination Generator

In [ ]:
@pytest.mark.asyncio
async def test_fetch_pages_generator_two_pages():
    """Test generator yields exactly 2 batches for 2-page dataset"""
    
    # Mock responses for 2 pages
    page1_response = Mock(spec=httpx.Response)
    page1_response.status_code = 200
    page1_response.json.return_value = {
        'data': {
            'users': {
                'nodes': [{'id': 1}, {'id': 2}],
                'pageInfo': {'hasNextPage': True, 'endCursor': 'cursor2'}
            }
        }
    }
    page1_response.raise_for_status = Mock()
    
    page2_response = Mock(spec=httpx.Response)
    page2_response.status_code = 200
    page2_response.json.return_value = {
        'data': {
            'users': {
                'nodes': [{'id': 3}, {'id': 4}],
                'pageInfo': {'hasNextPage': False, 'endCursor': None}
            }
        }
    }
    page2_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        # Return different responses for each call
        mock_client_instance.request.side_effect = [page1_response, page2_response]
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            batches = []
            async for batch in client.fetch_pages_generator(
                query_template='''
                    query($cursor: String) {
                        users(after: $cursor) {
                            nodes { id }
                            pageInfo { hasNextPage endCursor }
                        }
                    }
                ''',
                variables={'cursor': None},
                items_path=['data', 'users', 'nodes'],
                cursor_path=['data', 'users', 'pageInfo', 'endCursor'],
                has_next_path=['data', 'users', 'pageInfo', 'hasNextPage']
            ):
                batches.append(batch)
            
            # Verify exactly 2 batches
            assert len(batches) == 2
            assert batches[0] == [{'id': 1}, {'id': 2}]
            assert batches[1] == [{'id': 3}, {'id': 4}]

# Run test
await test_fetch_pages_generator_two_pages()
print("✅ Test passed: Fetch pages generator (2 pages)")

✅ Test passed: Fetch pages generator (2 pages)


In [ ]:
@pytest.mark.asyncio
async def test_cursor_extraction_and_update():
    """Test cursor is correctly extracted and updated between pages"""
    
    # Mock responses
    page1_response = Mock(spec=httpx.Response)
    page1_response.status_code = 200
    page1_response.json.return_value = {
        'data': {
            'users': {
                'nodes': [{'id': 1}],
                'pageInfo': {'endCursor': 'CURSOR_ABC'}
            }
        }
    }
    page1_response.raise_for_status = Mock()
    
    page2_response = Mock(spec=httpx.Response)
    page2_response.status_code = 200
    page2_response.json.return_value = {
        'data': {
            'users': {
                'nodes': [{'id': 2}],
                'pageInfo': {'endCursor': None}  # Last page
            }
        }
    }
    page2_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.side_effect = [page1_response, page2_response]
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            variables = {'cursor': None}
            batch_count = 0
            
            async for batch in client.fetch_pages_generator(
                query_template='query($cursor: String) { users(after: $cursor) { nodes { id } pageInfo { endCursor } } }',
                variables=variables,
                items_path=['data', 'users', 'nodes'],
                cursor_path=['data', 'users', 'pageInfo', 'endCursor']
            ):
                batch_count += 1
            
            # Verify 2 batches were processed
            assert batch_count == 2
            
            # Verify the second request was made with the extracted cursor
            assert mock_client_instance.request.call_count == 2
            
            # Check the variables in the second call included the cursor
            second_call_kwargs = mock_client_instance.request.call_args_list[1][1]
            second_call_json = second_call_kwargs.get('json', {})
            second_call_variables = second_call_json.get('variables', {})
            
            assert second_call_variables.get('cursor') == 'CURSOR_ABC', \
                f"Expected cursor 'CURSOR_ABC', got {second_call_variables.get('cursor')}"

# Run test
await test_cursor_extraction_and_update()
print("✅ Test passed: Cursor extraction and update")

✅ Test passed: Cursor extraction and update


In [ ]:
@pytest.mark.asyncio
async def test_fetch_pages_relay_with_variables():
    """Test fetch_pages_relay() passes through custom variables"""
    
    # Mock response
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'data': {
            'transactionsConnection': {
                'edges': [{'node': {'id': 1}}],
                'pageInfo': {'hasNextPage': False, 'endCursor': None}
            }
        }
    }
    mock_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            # Pass custom variables
            all_items = await client.fetch_pages_relay(
                query='''
                    query($first: Int, $after: String, $filter: Filter) {
                        transactionsConnection(first: $first, after: $after, filter: $filter) {
                            edges { node { id } }
                            pageInfo { hasNextPage endCursor }
                        }
                    }
                ''',
                connection_path="transactionsConnection",
                variables={"filter": {"status": "completed"}},
                page_size=50
            )
            
            # Verify custom variables were passed
            call_kwargs = mock_client_instance.request.call_args_list[0][1]
            request_json = call_kwargs.get('json', {})
            request_vars = request_json.get('variables', {})
            
            assert request_vars['filter'] == {"status": "completed"}
            assert request_vars['first'] == 50
            assert 'after' in request_vars

# Run test
await test_fetch_pages_relay_with_variables()
print("✅ Test passed: fetch_pages_relay() with custom variables")

In [ ]:
@pytest.mark.asyncio
async def test_fetch_pages_relay_max_pages():
    """Test fetch_pages_relay() respects max_pages limit"""
    
    # Mock response that always has more pages
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'data': {
            'itemsConnection': {
                'edges': [{'node': {'id': i}} for i in range(10)],
                'pageInfo': {'hasNextPage': True, 'endCursor': 'next_cursor'}
            }
        }
    }
    mock_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        # Return same response indefinitely (simulates infinite pagination)
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            # Limit to 3 pages
            all_items = await client.fetch_pages_relay(
                query='''
                    query($first: Int, $after: String) {
                        itemsConnection(first: $first, after: $after) {
                            edges { node { id } }
                            pageInfo { hasNextPage endCursor }
                        }
                    }
                ''',
                connection_path="itemsConnection",
                page_size=10,
                max_pages=3
            )
            
            # Should stop at 3 pages = 30 items
            assert len(all_items) == 30
            assert mock_client_instance.request.call_count == 3

# Run test
await test_fetch_pages_relay_max_pages()
print("✅ Test passed: fetch_pages_relay() respects max_pages limit")

In [ ]:
@pytest.mark.asyncio
async def test_fetch_pages_relay():
    """Test fetch_pages_relay() accumulates all pages"""
    
    # Mock responses for Relay-style pagination
    page1_response = Mock(spec=httpx.Response)
    page1_response.status_code = 200
    page1_response.json.return_value = {
        'data': {
            'transactionsConnection': {
                'edges': [
                    {'node': {'id': 1, 'amount': 100}},
                    {'node': {'id': 2, 'amount': 200}}
                ],
                'pageInfo': {'hasNextPage': True, 'endCursor': 'cursor_page2'}
            }
        }
    }
    page1_response.raise_for_status = Mock()
    
    page2_response = Mock(spec=httpx.Response)
    page2_response.status_code = 200
    page2_response.json.return_value = {
        'data': {
            'transactionsConnection': {
                'edges': [
                    {'node': {'id': 3, 'amount': 300}}
                ],
                'pageInfo': {'hasNextPage': False, 'endCursor': None}
            }
        }
    }
    page2_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.side_effect = [page1_response, page2_response]
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            # Test fetch_pages_relay
            all_items = await client.fetch_pages_relay(
                query='''
                    query($first: Int, $after: String) {
                        transactionsConnection(first: $first, after: $after) {
                            edges { node { id amount } }
                            pageInfo { hasNextPage endCursor }
                        }
                    }
                ''',
                connection_path="transactionsConnection",
                page_size=2
            )
            
            # Verify all items accumulated
            assert len(all_items) == 3
            assert all_items[0] == {'id': 1, 'amount': 100}
            assert all_items[1] == {'id': 2, 'amount': 200}
            assert all_items[2] == {'id': 3, 'amount': 300}

# Run test
await test_fetch_pages_relay()
print("✅ Test passed: fetch_pages_relay() accumulates all pages")

In [ ]:
@pytest.mark.asyncio
async def test_execute_graphql_one_liner():
    """Test standalone execute_graphql() function"""
    from fh_saas.utils_graphql import execute_graphql
    
    # Mock response
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'data': {'user': {'id': 1, 'name': 'Bob'}}
    }
    mock_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        # One-liner execution
        result = await execute_graphql(
            url="https://api.example.com/graphql",
            query="query { user { id name } }",
            bearer_token="test-token"
        )
        
        assert result == {'user': {'id': 1, 'name': 'Bob'}}

# Run test
await test_execute_graphql_one_liner()
print("✅ Test passed: execute_graphql() one-liner function")

In [ ]:
@pytest.mark.asyncio
async def test_execute_unified_method():
    """Test GraphQLClient.execute() returns data portion only"""
    
    # Mock response
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'data': {
            'users': [{'id': 1, 'name': 'Alice'}]
        },
        'extensions': {'traceId': 'abc123'}
    }
    mock_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            # execute() should return only the 'data' portion
            result = await client.execute('{ users { id name } }')
            
            assert result == {'users': [{'id': 1, 'name': 'Alice'}]}
            assert 'extensions' not in result  # Verify it's just data

# Run test
await test_execute_unified_method()
print("✅ Test passed: GraphQLClient.execute() unified method")

In [ ]:
@pytest.mark.asyncio
async def test_from_url_constructor():
    """Test GraphQLClient.from_url() convenience constructor"""
    
    # Mock response
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'data': {'__typename': 'Query'}
    }
    mock_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        # Test using from_url
        async with GraphQLClient.from_url(
            url="https://api.example.com/graphql",
            bearer_token="test-token-123"
        ) as gql:
            assert gql is not None
            assert gql._owns_api is True
            
            result = await gql.execute("query { __typename }")
            assert result == {'__typename': 'Query'}

# Run test
await test_from_url_constructor()
print("✅ Test passed: GraphQLClient.from_url() constructor")

## Test New Convenience Methods